## Reading table

In [1]:
import csv
import os
import numpy as np
from PIL import Image
import torch
import pandas as pd

train_csv_path = '/kaggle/input/cse-255-hw4-table/train.csv'
image_path = '/kaggle/input/cse-255-hw4-image/anon_images/'

train_csv_df = pd.read_csv(train_csv_path, index_col=0)
train_csv_df_urban = train_csv_df.loc[train_csv_df['urban']==True]
train_csv_df_rural = train_csv_df.loc[train_csv_df['urban']==False]
train_csv_df_rural

,filename,country,wealthpooled,urban,label,nl_mean
0,image14517.npz,6,-1.019361,False,0,-0.086633
2,image7407.npz,6,-1.143002,False,0,-0.141589
13,image18643.npz,6,-0.955533,False,0,-0.173862
19,image13449.npz,6,0.158699,False,1,0.913758
21,image5255.npz,6,-1.149296,False,0,-0.036045
...,...,...,...,...,...,...
19651,image13672.npz,8,0.284203,False,1,-0.216665
19652,image19234.npz,8,-0.183867,False,1,-0.186854
19659,image13328.npz,8,0.047057,False,1,-0.216885
19661,image2309.npz,8,0.244931,False,1,-0.207048


In [2]:
from sklearn.model_selection import train_test_split
train_rural_indices, valid_rural_indices = train_test_split(range(len(train_csv_df_rural)), train_size=0.9)

In [3]:
training_set = train_csv_df_rural.iloc[train_rural_indices]
validation_set = train_csv_df_rural.iloc[valid_rural_indices]

## Reading image

In [4]:
import os
import random
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from PIL import Image
# PIL_image = Image.fromarray(ndarray_image)

class WildsDataset(Dataset):    
    def __init__(self, csv_df, mode = "train", transform=None):
        self.mode = mode
        self.transform = transform
        self.data_list = []
        self.label = []
        
        for index, row in csv_df.iterrows():
            self.data_list.append(image_path+row['filename'])
            if self.mode != 'test':
                self.label.append(row['label'])
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image = np.load(self.data_list[idx])
        image = image.f.x
        image = torch.from_numpy(image)
#         print(type(image))
#         print(image)
        if self.transform is not None:
            image = self.transform(image)
        if self.mode == 'test': 
            return image
        label = torch.tensor(int(self.label[idx]))
            
        return image, label



## Define Transform

In [5]:
import torchvision.transforms.functional as TF
import random

class TrainTransform:
    """Rotate by one of the given angles."""

    def __init__(self):
        self.angles = [-270, -180, -90, 0, 90, 180, 270]
        self.maxsize = [i for i in range(256,272)]
        self.boolean = [True, False]

    def __call__(self, x):
        angle = random.choice(self.angles)
        size = random.choice(self.maxsize)
        image = x
        image = TF.rotate(image, angle) #Rotation 
        image = TF.resize(image,size) #Resize
        image = TF.center_crop(image, 224) #Center Crop
        if random.choice(self.boolean):
            image = TF.vflip(image)
        if random.choice(self.boolean):
            image = TF.hflip(image)
#         print(image.shape)
#         print(torch.mean(image, dim = (1, 2)).shape)
#         print(torch.std(image.to(torch.double), dim = (1, 2), unbiased = True))
#         image = TF.normalize(image, mean=torch.mean(image, dim = (1, 2)), std = [0.5]*8)
        return image

class TestTransform:
    """Rotate by one of the given angles."""

    def __init__(self):
        self.angles = [-270, -180, -90, 0, 90, 180, 270]
        self.maxsize = [i for i in range(256,272)]
        self.boolean = [True, False]

    def __call__(self, x):
        angle = random.choice(self.angles)
        size = random.choice(self.maxsize)
        image = x
#         image = TF.normalize(image, mean=torch.mean(image, dim = (1, 2)), std = [0.5]*8)
        return image
    

In [6]:
dataset_train = WildsDataset(training_set, mode='train', transform = TrainTransform())
dataset_val = WildsDataset(validation_set, mode='val', transform = TestTransform())

In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=128, shuffle=False)

## Define Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler
from torch.utils.data import TensorDataset
from torchvision.utils import make_grid
import torchvision.models as models

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        model_ft = models.resnet34(pretrained=True)
#         model_ft.aux_logits = False 
        model_ft.conv1 = nn.Conv2d(8, 64, kernel_size=7, stride=2, padding = 3,bias=False)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 128)
        
        self.pretrain_model = model_ft 
        self.category = nn.Sequential(
            nn.Linear(128,32),
            nn.Dropout(0.5),
            nn.Linear(32,8),
            nn.Dropout(0.5),
            nn.Linear(8,2)
        )
    def forward(self, x):
        x = self.pretrain_model(x)
#         print(type(x))
        category = self.category(x)
        
        return category

In [9]:
model = Net()
model = model.cuda()

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

## Freeze layers for fine tune

In [10]:
# i = 0
# for param in model.parameters():
#     if i<46:
#         param.requires_grad = False
#     i += 1

In [11]:
# for name,param in model.named_parameters():
#     if param.requires_grad == True:
#         print("\t",name)

## Criterion and Optimizer

In [12]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# criterion = nn.HingeEmbeddingLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01, momentum=0.9) # throw param into optimizer some_optimier(param, lr=...)
# optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-03, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
# optimizer = torch.optim.AdamW(params = model.parameters(), lr=1e-02, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.05, amsgrad=False)
# optimizer = torch.optim.RMSprop(params = model.parameters(), lr=1e-02, alpha = 0.9, momentum=0.9)
criterion = criterion.cuda()

## Train Function

In [13]:
def train(input_data, model, criterion, optimizer, transform = None):
    '''
    Argement:
    input_data -- iterable data, typr torch.utils.data.Dataloader is prefer
    model -- nn.Module, model contain forward to predict output
    criterion -- loss function, used to evaluate goodness of model
    optimizer -- optmizer function, method for weight updating
    '''
    model.train()
    loss_list = []
    total_count = 0
    acc_count = 0
    pbar = tqdm(input_data)
    for data in pbar:
        pbar.set_description("Training")
        images, labels = data[0].cuda(), data[1].cuda()
        if transform:
            images = transform(images)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() 
        
        _, predicted = torch.max(outputs.data, 1)
        total_count += labels.size(0) 
        acc_count += (predicted == labels).sum()  
        loss_list.append(loss.item())


    # Compute this epoch accuracy and loss
    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

## Validation Function

In [14]:
def val(input_data, model, criterion, transform = None):
    model.eval()
    
    loss_list = []
    total_count = 0
    acc_count = 0
    with torch.no_grad():
        pbar = tqdm(input_data)
        for data in pbar:
            pbar.set_description("Validation")
            images, labels = data[0].cuda(), data[1].cuda()
            if transform:
                transform(images)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, dim=1)
            total_count += labels.size(0)
            acc_count += (predicted == labels).sum()
            loss_list.append(loss.item())

    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

## Start training

In [15]:
!cp -r ../input/earlystopping/early-stopping-pytorch-master/* ./
from pytorchtools import EarlyStopping

In [16]:
max_epochs = 50
log_interval = 1 # print acc and loss in per log_interval time
patience = 5
early_stopping = EarlyStopping(patience=patience, verbose=True, path='checkpoint_rural.pt')
train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []

for epoch in range(1, max_epochs + 1):
    train_acc, train_loss = train(train_loader, model, criterion, optimizer)
    val_acc, val_loss = val(val_loader, model, criterion)

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)
    if epoch % log_interval == 0:
        print('=' * 20, 'Epoch', epoch, '=' * 20)
        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss))
        print('  Val Acc: {:.6f}   Val Loss: {:.6f}'.format(val_acc, val_loss))
    
    early_stopping(-val_acc, model)
#     if early_stopping.early_stop:
#         print("Early stopping")
#         break

Validation: 100%|██████████| 6/6 [00:17<00:00,  2.92s/it]


==================== Epoch 1 ====================
Train Acc: 0.603655 Train Loss: 0.665520
  Val Acc: 0.649171   Val Loss: 0.614975
Validation loss decreased (inf --> -0.649171).  Saving model ...


Validation: 100%|██████████| 6/6 [00:08<00:00,  1.36s/it]


==================== Epoch 2 ====================
Train Acc: 0.670455 Train Loss: 0.612412
  Val Acc: 0.683702   Val Loss: 0.602995
Validation loss decreased (-0.649171 --> -0.683702).  Saving model ...


Validation: 100%|██████████| 6/6 [00:08<00:00,  1.39s/it]


...

==================== Epoch 48 ====================
Train Acc: 0.882217 Train Loss: 0.292760
  Val Acc: 0.715470   Val Loss: 0.821842
EarlyStopping counter: 34 out of 5


Validation: 100%|██████████| 6/6 [00:08<00:00,  1.41s/it]


==================== Epoch 49 ====================
Train Acc: 0.887592 Train Loss: 0.284173
  Val Acc: 0.712707   Val Loss: 0.893222
EarlyStopping counter: 35 out of 5


Validation: 100%|██████████| 6/6 [00:08<00:00,  1.44s/it]

==================== Epoch 50 ====================
Train Acc: 0.886671 Train Loss: 0.270493
  Val Acc: 0.691989   Val Loss: 0.890965
EarlyStopping counter: 36 out of 5
